In [ ]:
from google.colab import auth
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import *
import numpy as np
import pickle

auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('Commit Classification Dataset').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows)
df = df.iloc[1:, 3 : ]

# New Section

In [ ]:
data_model1 = df.drop(df.iloc[:, 1:49],axis = 1).to_numpy()

input_data_keyword = data_model1[:,1:]
input_data_keyword = pd.DataFrame(input_data_keyword).apply(pd.to_numeric).to_numpy()
output_data_keyword = data_model1[:,:1]

data_model2 = df.to_numpy()

input_data_combined = data_model2[:,1:]
input_data_combined = pd.DataFrame(input_data_combined).apply(pd.to_numeric).to_numpy()
output_data_combined = data_model2[:,:1]


In [ ]:
x_train_keyword,x_test_keyword,y_train_keyword,y_test_keyword=train_test_split(input_data_keyword,output_data_keyword,test_size=0.15)
x_train_combined,x_test_combined,y_train_combined,y_test_combined=train_test_split(input_data_combined,output_data_combined,test_size=0.15)

x_test_combined[np.isnan(x_test_combined)] = np.nanmean(x_test_combined)

kfold = KFold(n_splits=10, shuffle=True, random_state=1)

clfKeyWord = RandomForestClassifier(n_estimators=10, max_depth=6, n_jobs=1, verbose=0)
clfCombined = RandomForestClassifier(n_estimators=10, max_depth=6, n_jobs=1, verbose=0)

for train_index, test_index in kfold.split(x_train_keyword):
  x_train, x_test = x_train_keyword[train_index], x_train_keyword[test_index]
  y_train, y_test = y_train_keyword[train_index], y_train_keyword[test_index]
  x_train[np.isnan(x_train)]= np.nanmean(x_train)
  x_test[np.isnan(x_test)]= np.nanmean(x_test)
  clfKeyWord.fit(x_train, y_train)
  keyword_model = pickle.dumps(clfKeyWord)

for train_index, test_index in kfold.split(x_train_combined):
  x_train, x_test = x_train_combined[train_index], x_train_combined[test_index]
  y_train, y_test = y_train_combined[train_index], y_train_combined[test_index]
  x_train[np.isnan(x_train)]= np.nanmean(x_train)
  x_test[np.isnan(x_test)]= np.nanmean(x_test)
  clfCombined.fit(x_train, y_train)
  combined_model = pickle.dumps(clfCombined)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()
/usr/loc

In [ ]:
loaded_model_combined = pickle.loads(combined_model)
loaded_model_keyword = pickle.loads(keyword_model)

y_predict = []
for x in x_test_combined:
  if ~np.all(x[48:] == 0, axis=0) : 
    y_predict.append(loaded_model_keyword.predict([x[48:]]))
  else :
    y_predict.append(loaded_model_combined.predict([x]))

print("Accuracy = " + str(accuracy_score(y_test_combined, y_predict)*100)+"%\n")
print("\nCohen Kappa Score = " + str(cohen_kappa_score(y_test_combined, y_predict)*100)+"%\n")
print(classification_report(y_test_combined,y_predict))

Accuracy = 71.67630057803468%


Cohen Kappa Score = 56.164029372220504%

              precision    recall  f1-score   support

                   1.00      1.00      1.00         3
           a       0.76      0.69      0.72        36
           c       0.68      0.85      0.75        72
           p       0.74      0.56      0.64        62

    accuracy                           0.72       173
   macro avg       0.80      0.78      0.78       173
weighted avg       0.72      0.72      0.71       173

